# Cuadernillo — **Opción B**: Máximo Clique con SA puro-Python (sin dimod/neal)

**Contenido:** setup, Paley(q) primo, QUBO libre y con tamaño fijado, SA puro-Python, tests en q=17,29.

In [ ]:

# Setup minimal
try:
    import networkx as nx; print("NetworkX:", nx.__version__)
except Exception:
    %pip -q install networkx
    import networkx as nx; print("NetworkX:", nx.__version__)


In [ ]:

import networkx as nx
def _is_probable_prime(n:int)->bool:
    if n<2: return False
    small=[2,3,5,7,11,13,17,19,23,29]
    for p in small:
        if n==p: return True
        if n%p==0: return n==p
    def mr_check(a,s,d,n):
        x=pow(a,d,n)
        if x in (1,n-1): return True
        for _ in range(s-1):
            x=(x*x)%n
            if x==n-1: return True
        return False
    d,s=n-1,0
    while d%2==0: d//=2; s+=1
    for a in [2,325,9375,28178,450775,9780504,1795265022]:
        if a % n == 0: continue
        if not mr_check(a,s,d,n): return False
    return True

def paley_graph_prime(q:int)->nx.Graph:
    if q%4!=1: raise ValueError("q ≡ 1 (mod 4)")
    if not _is_probable_prime(q): raise ValueError("Usa q primo (17,29,37,...)")
    G=nx.Graph(); G.add_nodes_from(range(q))
    residues={(x*x)%q for x in range(1,q)}
    for u in range(q):
        for v in range(u+1,q):
            if (v-u)%q in residues: G.add_edge(u,v)
    return G
G17=paley_graph_prime(17)
print("Paley(17): n, k =", G17.number_of_nodes(), next(iter(dict(G17.degree()).values())))


In [ ]:

import networkx as nx
def build_qubo_max_clique(G, M=1.0, beta=2.0):
    n=G.number_of_nodes(); nodes=list(G.nodes()); idx={u:i for i,u in enumerate(nodes)}
    h=[-M]*n
    comp=nx.complement(G)
    J=[[0.0]*n for _ in range(n)]
    for u,v in comp.edges():
        i,j=idx[u],idx[v]; J[i][j]+=beta; J[j][i]+=beta
    return nodes,h,J

def build_qubo_fixed_size(G, m, beta=2.0, eta=10.0):
    nodes=list(G.nodes()); idx={u:i for i,u in enumerate(nodes)}; n=len(nodes)
    comp=nx.complement(G)
    h=[eta*(1-2*m)]*n
    J=[[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            coeff=2.0*eta
            if comp.has_edge(nodes[i],nodes[j]): coeff+=beta
            J[i][j]=J[j][i]=coeff
    return nodes,h,J


In [ ]:

import random, math
def energy(x,h,J):
    n=len(x); e=sum(h[i]*x[i] for i in range(n))
    for i in range(n):
        if x[i]:
            Ji=J[i]
            e+=sum(Ji[j] for j in range(i+1,n) if x[j] and Ji[j])
    return e

def delta_flip(i,x,h,J):
    n=len(x); Ji=J[i]
    if x[i]==0:
        return h[i]+sum(Ji[j] for j in range(n) if j!=i and x[j] and Ji[j])
    else:
        return -h[i]-sum(Ji[j] for j in range(n) if j!=i and x[j] and Ji[j])

def sa_qubo(h,J,steps=150000,T0=1.5,Tf=1e-3,seed=7):
    random.seed(seed); n=len(h); x=[0]*n; e=energy(x,h,J); best_x=x[:]; best_e=e
    for t in range(1,steps+1):
        T=T0*(Tf/T0)**(t/steps); i=random.randrange(n); dE=delta_flip(i,x,h,J)
        if dE<=0 or random.random()<math.exp(-dE/max(T,1e-12)):
            x[i]^=1; e+=dE
            if e<best_e: best_e, best_x=e, x[:]
    return best_x, best_e

def is_clique(G,S):
    H=G.subgraph(S); t=len(S)
    return H.number_of_edges()==t*(t-1)//2
def clique_from_state(x,nodes):
    return [nodes[i] for i,v in enumerate(x) if v==1]


In [ ]:

# Máximo cliqué para Paley(17) y Paley(29)
for q in (17,29):
    G=paley_graph_prime(q)
    nodes,h,J=build_qubo_max_clique(G, M=1.0, beta=2.0)
    x,e=sa_qubo(h,J,steps=150000,T0=1.5,Tf=1e-3,seed=7)
    S=clique_from_state(x,nodes)
    print(f"[SA] Paley({q}): |S|={len(S)}, es_clique={is_clique(G,S)}, S={sorted(S)}, E={e:.3f}")


In [ ]:

# Certificación m=3 y m=4 en Paley(17)
G=paley_graph_prime(17)
for m in (3,4):
    nodes,h,J=build_qubo_fixed_size(G, m=m, beta=2.0, eta=10.0)
    x,e=sa_qubo(h,J,steps=180000,T0=2.0,Tf=1e-3,seed=11)
    S=clique_from_state(x,nodes)
    print(f"[SA fixed-m] m={m} -> |S|={len(S)}, es_clique={is_clique(G,S)}, S={sorted(S)}, E={e:.3f}")
